### Sentiment Analysis with the list of positive and negative words and set intersection
#### Accuracy - 0.63

In [1]:
import numpy as np
import re
import pandas as pd
from nltk import wordpunct_tokenize
import string

In [4]:
pd.set_option('display.max_columns', None)

In [ ]:
test_data = pd.read_csv("C:/path_for_test_data/dataset2_final_eval.csv",skipinitialspace=True)

#### List of negative and positive words

In [3]:
negative = pd.read_csv("C:/Path_for_file_with_list_of_negative_words/negative.csv", header = None)
positive = pd.read_csv("C:/Path_for_file_with_list_of_positive_words/positive.csv", header = None)

In [9]:
negative.head()

,0
0,2-faced
1,2-faces
2,abnormal
3,abolish
4,abominable


In [10]:
positive.head()

,0
0,a+
1,abound
2,abounds
3,abundance
4,abundant


In [5]:
test_data["sentiment"] = np.nan
test_data.head()

,id,text,sentiment
0,0,@USER ahahahaha! ghostbusters degree! amazing,NaN
1,1,this poor lady has lost here dog WEBSITE2018,NaN
2,2,@USER i've never seen them &quot;in the fles...,NaN
3,3,@USER hey there donnie! hope all is well. if ...,NaN
4,4,i've totally forgotten that i have a twitter a...,NaN


In [6]:
sentiment_data = [0,1]

#### Removed the nametags, hashtags, punctuations and smileys from the twitter feed for test file

In [7]:
test_data["text"] = test_data["text"].str.replace("(@[A-Za-z0-9]+)","")
test_data["text"] = test_data["text"].str.replace("(#[A-Za-z0-9]+)","")
test_data["text"] = test_data["text"].str.replace("[{}]".format(string.punctuation), '')
test_data["text"] = test_data["text"].str.strip()
test_data["text"] = test_data["text"].str.replace("[^\w\s#@/:%.,_-]", "", flags=re.UNICODE)
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [8]:
test_data.head()

,id,text,sentiment
0,0,ahahahaha ghostbusters degree amazing,NaN
1,1,this poor lady has lost here dog WEBSITE2018,NaN
2,2,ive never seen them quotin the fleshquot but i...,NaN
3,3,hey there donnie hope all is well if youre in ...,NaN
4,4,ive totally forgotten that i have a twitter a ...,NaN


#### From the neg/pos words dataset, create list of words to help in the test data detection 

In [11]:
negative = negative.dropna()
negative = negative[:].values.tolist()
negative = [item for sublist in negative for item in sublist]

positive = positive.dropna()
positive = positive[:].values.tolist()
positive = [item for sublist in positive for item in sublist]

In [12]:
sentiment_language = int

#### Tokenize each twitter feed from the test data and find an intersection with the set of identified words attributed tothe sentiment in [11] command.

#### Then find the ratio of words of each sentiment in the twitter feed, then attribute the tweet to the sentiment (positive or negative) with highest ratio

In [13]:
for index, row in test_data.iterrows():
     
    f = wordpunct_tokenize(row["text"])
    words = (word.lower() for word in f)
    sentiment_ratio = {}

    sentiment = {0 : negative, 1 : positive}

    stopwordSet_negative = set(negative)
    stopwordSet_positive = set(positive)

    word_set = set(words)

    common_elements_negative = word_set.intersection(stopwordSet_negative)
    common_elements_positive = word_set.intersection(stopwordSet_positive)

    sentiment_ratio[0] = len(common_elements_negative)
    sentiment_ratio[1] = len(common_elements_positive)

    sentiment_language = str( max(sentiment_ratio, key = sentiment_ratio.get))
    test_data.sentiment.iloc[index] = sentiment_language

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### Test Data with identified sentiment with the help of training data

In [14]:
test_data.head()

,id,text,sentiment
0,0,ahahahaha ghostbusters degree amazing,1
1,1,this poor lady has lost here dog WEBSITE2018,0
2,2,ive never seen them quotin the fleshquot but i...,0
3,3,hey there donnie hope all is well if youre in ...,1
4,4,ive totally forgotten that i have a twitter a ...,0


In [15]:
result = test_data[["id","sentiment"]]
result.head()

,id,sentiment
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0
